# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: xx

Group Members:
- Name (NIM)
- Name (NIM)
- ...

## Import Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np
import re
import string


# Import other libraries if needed

## Import Dataset

In [55]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public

df = pd.read_csv('train.csv')
df.head()

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


In [56]:
print(df[['URL', 'TLDLength', 'NoOfSubDomain']])
print(df.columns)

                                         URL  TLDLength  NoOfSubDomain
0                  https://www.northcm.ac.th        NaN            2.0
1                        http://uqr.to/1il1z        2.0            0.0
2       https://www.woolworthsrewards.com.au        NaN            2.0
3                                        NaN        NaN            1.0
4                                        NaN        NaN            1.0
...                                      ...        ...            ...
140399                                   NaN        3.0            1.0
140400   https://www.greenmountainenergy.com        3.0            NaN
140401       https://www.leadcastingcall.com        3.0            1.0
140402                https://www.fedarb.com        NaN            NaN
140403           https://www.risenenergy.com        3.0            1.0

[140404 rows x 3 columns]
Index(['id', 'FILENAME', 'URL', 'URLLength', 'Domain', 'DomainLength',
       'IsDomainIP', 'TLD', 'CharContinuationRate'

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [57]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
# train_set, val_set = ...
no_label = df.drop(columns=['label'])
label_only = df['label']

X_train, X_val, y_train, y_val = train_test_split(no_label, label_only, test_size=0.2, random_state=42, stratify=label_only)

print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")

Training set size: 112323
Validation set size: 28081


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [58]:
#Domain extract from URL
def extract_domain_from_url(url):
    if isinstance(url, str):
        # Use regex to extract domain from the URL
        match = re.search(r'https?://(?:www\.)?([^/]+)', url)
        if match:
            return match.group(1)
    return None

df.loc[df['Domain'].isna() & df['URL'].notna(), 'Domain'] = df['URL'].apply(extract_domain_from_url)

domain_null = df[df['Domain'].isna()]
domain_null_count = df['Domain'].isna().sum()

# Display the updated dataframe
print(domain_null_count)
print(domain_null)

21760
            id    FILENAME  URL  URLLength Domain  DomainLength  IsDomainIP  \
3            6         NaN  NaN       31.0    NaN           NaN         NaN   
33          71  616446.txt  NaN       35.0    NaN           NaN         0.0   
36          75         NaN  NaN       27.0    NaN          20.0         0.0   
47          94  547586.txt  NaN        NaN    NaN          12.0         NaN   
65         121         NaN  NaN       23.0    NaN          17.0         0.0   
...        ...         ...  ...        ...    ...           ...         ...   
140364  235723  434252.txt  NaN       21.0    NaN          14.0         0.0   
140376  235750         NaN  NaN        NaN    NaN          20.0         0.0   
140377  235752         NaN  NaN        NaN    NaN          31.0         0.0   
140380  235757         NaN  NaN        NaN    NaN          24.0         0.0   
140398  235787  856116.txt  NaN       26.0    NaN          19.0         0.0   

         TLD  CharContinuationRate  TLDLegiti

In [59]:
#Extract TLD dari domain AND TLD length from TLD

def extract_tld(domain):
    if isinstance(domain, str):
        return domain.split('.')[-1]  # Return the last part after splitting by '.'
    return None

df.loc[df['TLD'].isna() & df['Domain'].notna(), 'TLD'] = df['Domain'].apply(extract_tld)

tld_null_count = df['TLD'].isna().sum()

#TLDLength
df['TLDLength'] = df.apply(lambda row: len(row['TLD']) if pd.notna(row['TLD']) and pd.isna(row['TLDLength']) else row['TLDLength'], axis=1)
print(df[['TLD', 'TLDLength']].head())

   TLD  TLDLength
0   th        2.0
1   to        2.0
2   au        2.0
3  com        3.0
4  com        3.0


In [60]:
# Auto fill untuk feature Numeric yang menyangkut URL

def analyze_url(row):
    if pd.isna(row['URL']):  
        return row  
    
    url = row['URL']
    url_length = len(url)
    letters = sum(1 for char in url if char.isalpha())
    digits = sum(1 for char in url if char.isdigit())
    equals = url.count('=')
    q_marks = url.count('?')
    ampersands = url.count('&')
    special_chars = sum(1 for char in url if char in string.punctuation and char not in ['=', '?', '&'])
    
    row['NoOfLettersInURL'] = letters
    row['LetterRatioInURL'] = letters / url_length if url_length > 0 else 0
    row['NoOfDegitsInURL'] = digits
    row['DegitRatioInURL'] = digits / url_length if url_length > 0 else 0
    row['NoOfEqualsInURL'] = equals
    row['NoOfQMarkInURL'] = q_marks
    row['NoOfAmpersandInURL'] = ampersands
    row['NoOfOtherSpecialCharsInURL'] = special_chars
    row['SpacialCharRatioInURL'] = (letters + digits + special_chars) / url_length if url_length > 0 else 0
    row['IsHTTPS'] = 1 if url.lower().startswith('https://') else 0
    
    return row

df = df.apply(analyze_url, axis=1)

In [61]:
missing_data = df.isnull().sum().reset_index()
missing_data.columns = ['Column', 'total missing']
display(missing_data)

,Column,total missing
0,id,0
1,FILENAME,57532
2,URL,43487
3,URLLength,60639
4,Domain,21760
5,DomainLength,46319
6,IsDomainIP,42130
7,TLD,7019
8,CharContinuationRate,48042
9,TLDLegitimateProb,52873


In [53]:
#Handling string in features yang harusnya numeric

for i in df.select_dtypes(include =[np.number]).columns:
    nonNumeric = pd.to_numeric(df[i], errors='coerce').isna() & df[i].notna()
    if nonNumeric.any():
        print(f"Column {i} has non-numeric values:")
        print(df[nonNumeric][i])
        print("\n")
        df[i] = pd.to_numeric(df[i], errors='coerce')


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [31]:
def impute_outliers_with_condition(df, feature, method='median'):
    if feature in ['LetterRatioInURL', 'DegitRatioInURL']:
        condition = df['URL'].notna()
    else:
        condition = pd.Series([False] * len(df))
    
    if method == 'mean':
        df.loc[~condition, feature] = df.loc[~condition, feature].fillna(df[feature].mean())
    elif method == 'median':
        df.loc[~condition, feature] = df.loc[~condition, feature].fillna(df[feature].median())
    elif method == 'boundary':
        lower_bound = df[feature].quantile(0.05)
        upper_bound = df[feature].quantile(0.95)
        df.loc[~condition, feature] = df.loc[~condition, feature].clip(lower=lower_bound, upper=upper_bound)
    return df

imputation_features = ['TLDLegitimateProb', 'LetterRatioInURL', 'DegitRatioInURL']

for feature in imputation_features:
    df = impute_outliers_with_condition(df, feature, method='median')

print(df.describe())


                  id     URLLength  DomainLength    IsDomainIP  \
count  140404.000000  79765.000000  94085.000000  98274.000000   
mean   117682.632746     27.701473     19.619387      0.000488   
std     68122.005080     23.140676      5.833808      0.022095   
min         1.000000     14.000000      4.000000      0.000000   
25%     58689.500000     23.000000     16.000000      0.000000   
50%    117421.500000     26.000000     19.000000      0.000000   
75%    176724.250000     30.000000     23.000000      0.000000   
max    235795.000000   4054.000000     93.000000      1.000000   

       CharContinuationRate  TLDLegitimateProb   URLCharProb      TLDLength  \
count          92362.000000      140404.000000  88333.000000  138051.000000   
mean               0.918007           0.369904      0.059286       2.729911   
std                0.159142           0.229171      0.008063       0.532136   
min                0.000000           0.000000      0.001229       2.000000   
25%       

In [32]:
#Clipping

clipping_features = [
    'URLLength', 'NoOfLettersInURL', 'NoOfDegitsInURL', 'NoOfEqualsInURL',
    'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
    'LargestLineLength', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef',
    'NoOfEmptyRef', 'NoOfExternalRef'
]

def clip_outliers(df, feature, lower_percentile=0.05, upper_percentile=0.95):
    lower_bound = df[feature].quantile(lower_percentile)
    upper_bound = df[feature].quantile(upper_percentile)
    df[feature] = df[feature].clip(lower=lower_bound, upper=upper_bound)
    return df

for i in clipping_features:
    df = clip_outliers(df, feature, lower_percentile=0.05, upper_percentile=0.95)

print(df.describe())

                  id     URLLength  DomainLength    IsDomainIP  \
count  140404.000000  79765.000000  94085.000000  98274.000000   
mean   117682.632746     27.701473     19.619387      0.000488   
std     68122.005080     23.140676      5.833808      0.022095   
min         1.000000     14.000000      4.000000      0.000000   
25%     58689.500000     23.000000     16.000000      0.000000   
50%    117421.500000     26.000000     19.000000      0.000000   
75%    176724.250000     30.000000     23.000000      0.000000   
max    235795.000000   4054.000000     93.000000      1.000000   

       CharContinuationRate  TLDLegitimateProb   URLCharProb      TLDLength  \
count          92362.000000      140404.000000  88333.000000  138051.000000   
mean               0.918007           0.369904      0.059286       2.729911   
std                0.159142           0.229171      0.008063       0.532136   
min                0.000000           0.000000      0.001229       2.000000   
25%       

In [33]:
#Transformation untuk feature CharContinuationRate, ObfuscationRatio, SpacialCharRatioInURL, LineOfCode, DomainTitleMatchScore, URLTitleMatchScore

transformation_features = [ 'CharContinuationRate', 'ObfuscationRatio', 'SpacialCharRatioInURL', 'LineOfCode', 'DomainTitleMatchScore', 'URLTitleMatchScore']
def transformation(df, feature, method='log'):
    if method == 'log':
        df[feature] = np.log1p(df[feature]) 
    elif method == 'sqrt':
        df[feature] = np.sqrt(df[feature])
    elif method == 'power':
        df[feature] = np.power(df[feature], 0.5)

    return df

for feature in transformation_features:
    df = transformation(df, feature, 'log')

print(df[['CharContinuationRate', 'ObfuscationRatio', 'SpacialCharRatioInURL', 'LineOfCode', 'DomainTitleMatchScore', 'URLTitleMatchScore']].describe())

       CharContinuationRate  ObfuscationRatio  SpacialCharRatioInURL  \
count          92362.000000      75806.000000          120989.000000   
mean               0.647400          0.000027               0.565066   
std                0.090948          0.001589               0.257034   
min                0.000000          0.000000               0.000000   
25%                0.648695          0.000000               0.693147   
50%                0.693147          0.000000               0.693147   
75%                0.693147          0.000000               0.693147   
max                0.693147          0.192272               0.693147   

         LineOfCode  DomainTitleMatchScore  URLTitleMatchScore  
count  71251.000000           90407.000000        88188.000000  
mean       6.753832               3.289043            3.304264  
std        1.478084               2.080185            2.071877  
min        1.098612               0.000000            0.000000  
25%        6.248043       

In [34]:
#Modelling 

modelling_features = ['NoOfSubDomain', 'NoOfLettersInURL', 'NoOfDegitsInURL', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']

def modellingOutlier(df, feature, method='boundary', contamination=0.01):
    model = IsolationForest(contamination=contamination, random_state=42)
    df['outlier'] = model.fit_predict(df[[feature]])


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

Number of rows before duplicate remove
(140404, 56)
Number of rows after duplicate remove
(140404, 56)


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [ ]:
#Buat 2 feature baru
df['URLComplexityScore'] = df['LetterRatioInURL'] + df['DegitRatioInURL'] + df['SpacialCharRatioInURL']
df['SubdomainLengthRatio'] = df['DomainLength'] / (df['NoOfSubDomain'] + 1) 
df['CombinedTitleMatchScore'] = df['DomainTitleMatchScore'] * df['URLTitleMatchScore']
df['SpecialCharDensity'] = df['NoOfOtherSpecialCharsInURL'] / (df['URLLength'] + 1)     

In [63]:
#Encoding TLD dan isHTTPS
tld_frequency = df['TLD'].value_counts(normalize=True)
df['TLDEncoded'] = df['TLD'].map(tld_frequency)
df = pd.get_dummies(df, columns=['TLD'], drop_first=True)


In [65]:
#Transformasi untuk features yang skew parah

skewed_features = ['LineOfCode', 'LargestLineLength']
for feature in skewed_features:
    df[feature] = np.log1p(df[feature])

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [ ]:
class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, impute_method='median', clip_percentiles=(0.05, 0.95)):
        """
        Initialize the preprocessor with specific parameters.
        Args:
        - impute_method (str): Method for imputing missing values ('mean', 'median').
        - clip_percentiles (tuple): Percentiles for clipping (lower, upper).
        """
        self.impute_method = impute_method
        self.clip_percentiles = clip_percentiles
        self.imputation_values = {}

    def fit(self, X, y=None):
        """
        Compute the values needed for imputation and clipping.
        Args:
        - X (pd.DataFrame): Input features.
        - y: Target variable (not used, included for compatibility).
        Returns:
        - self
        """
        for col in X.select_dtypes(include=[np.number]).columns:
            if self.impute_method == 'mean':
                self.imputation_values[col] = X[col].mean()
            elif self.impute_method == 'median':
                self.imputation_values[col] = X[col].median()
        return self

    def transform(self, X, y=None):
        """
        Apply imputation and clipping to the data.
        Args:
        - X (pd.DataFrame): Input features.
        - y: Target variable (not used, included for compatibility).
        Returns:
        - X_transformed (pd.DataFrame): Preprocessed features.
        """
        X = X.copy()
        
        # Impute missing values
        for col, value in self.imputation_values.items():
            X[col] = X[col].fillna(value)
        
        # Clip numeric columns to specified percentiles
        for col in X.select_dtypes(include=[np.number]).columns:
            lower_bound = X[col].quantile(self.clip_percentiles[0])
            upper_bound = X[col].quantile(self.clip_percentiles[1])
            X[col] = X[col].clip(lower=lower_bound, upper=upper_bound)
        
        return X

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [38]:
# Write your code here

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [39]:
# Write your code here

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [40]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [41]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [42]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [43]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [44]:
# Type your code here

## B. Naive Bayes

In [45]:
# Type your code here

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [46]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [47]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`